# Assembly Line Balancing with Sequence-Dependent Setups (MILP)

Dieses Notebook modelliert ein Assembly Line Balancing (ALB) Problem mit sequenzabhängigen Rüstzeiten in Python mithilfe von Gurobi. Die Struktur orientiert sich an der Datei `2021-10-14_SALBP2_solution.ipynb`, nutzt aber ein exaktes MILP-Modell anstelle der heuristischen Ansätze aus dem Paper *"New solution approaches for balancing assembly lines with setup times"*.

## 1) Bibliotheken laden

In [ ]:
from gurobipy import GRB, Model, quicksum
import pandas as pd

## 2) Beispieldaten definieren

Die Daten umfassen Arbeitsaufgaben, Bearbeitungszeiten, eine Vorgängerstruktur und Produktfamilien, die die sequenzabhängigen Rüstzeiten bestimmen. Die Rüstzeiten orientieren sich an den Familienwechseln aus dem Paper; innerhalb einer Familie ist die Rüstzeit niedriger als bei einem Familienwechsel.

In [ ]:
# Bearbeitungszeiten (Minuten pro Aufgabe)
processing_time = {
    "A": 5,
    "B": 3,
    "C": 4,
    "D": 6,
    "E": 2,
    "F": 5,
    "G": 3,
    "H": 4,
    "I": 3,
    "J": 2,
}

# Vorgängerbeziehungen (i -> j bedeutet i muss vor j abgeschlossen sein)
precedence = [
    ("A", "C"), ("A", "D"), ("B", "D"), ("B", "E"),
    ("C", "F"), ("D", "G"), ("E", "G"), ("F", "H"),
    ("G", "I"), ("H", "J"), ("I", "J"),
]

# Produktfamilien (für sequenzabhängige Rüstzeiten)
family = {
    "A": "chassis",
    "B": "chassis",
    "C": "electronics",
    "D": "electronics",
    "E": "electronics",
    "F": "mechanics",
    "G": "mechanics",
    "H": "finishing",
    "I": "finishing",
    "J": "finishing",
}

# Rüstzeiten beim Wechsel zwischen Produktfamilien (Minuten)
setup_family = {
    "chassis": {"chassis": 0, "electronics": 2, "mechanics": 3, "finishing": 4},
    "electronics": {"chassis": 2, "electronics": 0.5, "mechanics": 1.5, "finishing": 3},
    "mechanics": {"chassis": 3, "electronics": 1.5, "mechanics": 0, "finishing": 2},
    "finishing": {"chassis": 4, "electronics": 3, "mechanics": 2, "finishing": 0},
}

# Stationen und Zykluszeit (C)
stations = list(range(1, 5))  # maximal 4 Stationen
cycle_time = 12

# Ableitung der paarweisen Rüstzeiten zwischen Aufgaben
tasks = list(processing_time.keys())
setup_time = {(i, j): setup_family[family[i]][family[j]] for i in tasks for j in tasks if i != j}

## 3) Modell erstellen

In [ ]:
model = Model("ALB_sequence_dependent_setups")

# Entscheidungsvariablen
x = model.addVars(tasks, stations, vtype=GRB.BINARY, name="assign")  # Aufgabe i an Station s?
y = model.addVars(stations, vtype=GRB.BINARY, name="use")            # Station wird verwendet?
start = model.addVars(tasks, lb=0, name="start")                     # Startzeit (innerhalb Zyklus)
pos = model.addVars(tasks, vtype=GRB.CONTINUOUS, name="station_index")

# Sequenzvariablen: Aufgabe i vor Aufgabe j auf Station s
z = model.addVars([(i, j, s) for i in tasks for j in tasks for s in stations if i != j],
                  vtype=GRB.BINARY, name="order")

# Hilfsgrößen
big_m = cycle_time

# 1) Jede Aufgabe genau einer Station zuordnen
for i in tasks:
    model.addConstr(quicksum(x[i, s] for s in stations) == 1, name=f"assign_{i}")

# 2) Station wird genutzt, wenn mindestens eine Aufgabe zugeordnet ist
for s in stations:
    for i in tasks:
        model.addConstr(x[i, s] <= y[s], name=f"link_use_{i}_{s}")

# 3) Stationsindex jeder Aufgabe (für Präzedenzbeziehungen)
for i in tasks:
    model.addConstr(pos[i] == quicksum(s * x[i, s] for s in stations), name=f"station_index_{i}")

# 4) Präzedenz: Vorgängerstation darf nicht nach Nachfolger kommen
for i, j in precedence:
    model.addConstr(pos[i] <= pos[j], name=f"precedence_station_{i}_{j}")

# 5) Sequenzierung innerhalb einer Station
for i in tasks:
    for j in tasks:
        if i == j:
            continue
        for s in stations:
            # z[i,j,s] und z[j,i,s] erzwingen eine Reihenfolge, falls beide Aufgaben auf Station s liegen
            model.addConstr(z[i, j, s] + z[j, i, s] == x[i, s] + x[j, s] - 1, name=f"order_pair_{i}_{j}_{s}")
            model.addConstr(z[i, j, s] <= x[i, s], name=f"order_link_i_{i}_{j}_{s}")
            model.addConstr(z[i, j, s] <= x[j, s], name=f"order_link_j_{i}_{j}_{s}")

            # Startzeiten mit Big-M an Sequenz koppeln
            model.addConstr(
                start[j] >= start[i] + processing_time[i] + setup_time[i, j] - big_m * (1 - z[i, j, s]),
                name=f"time_order_{i}_{j}_{s}")

# 6) Jede Aufgabe muss innerhalb der Zykluszeit abgeschlossen werden
for i in tasks:
    for s in stations:
        model.addConstr(start[i] + processing_time[i] <= cycle_time + big_m * (1 - x[i, s]),
                        name=f"cycle_{i}_{s}")

# 7) Ziel: Anzahl genutzter Stationen minimieren
model.setObjective(quicksum(y[s] for s in stations), GRB.MINIMIZE)

## 4) Modell lösen

In [ ]:
model.Params.OutputFlag = 0  # Ausgabe unterdrücken
model.optimize()

print(f"Status: {model.Status}")
print(f"Optimale Stationenzahl: {model.ObjVal}")

## 5) Lösung interpretieren

Wir rekonstruieren die Reihenfolge je Station anhand der `z`-Variablen und stellen Startzeiten sowie Familienwechsel dar.

In [ ]:
if model.Status == GRB.OPTIMAL:
    assignment = {i: [s for s in stations if x[i, s].X > 0.5][0] for i in tasks}
    starts = {i: start[i].X for i in tasks}

    station_sequences = {}
    for s in stations:
        tasks_on_s = [i for i in tasks if assignment[i] == s]
        if not tasks_on_s:
            continue
        # Graph aus Ordnung herstellen
        edges = {(i, j) for i in tasks_on_s for j in tasks_on_s if i != j and z[i, j, s].X > 0.5}
        before = {j: i for (i, j) in edges}
        # Kopf finden (der, der nie als j auftaucht)
        candidates = set(tasks_on_s) - {j for (_, j) in edges}
        ordered = []
        while candidates:
            current = candidates.pop()
            ordered.append(current)
            nxt = before.get(current)
            while nxt:
                ordered.append(nxt)
                nxt = before.get(nxt)
        # Sortiere zur Sicherheit nach Startzeit
        ordered = sorted(tasks_on_s, key=lambda t: starts[t])
        station_sequences[s] = ordered

    rows = []
    for s, seq in station_sequences.items():
        for i in seq:
            rows.append({
                "Station": s,
                "Aufgabe": i,
                "Familie": family[i],
                "Start": round(starts[i], 2),
                "Ende": round(starts[i] + processing_time[i], 2),
                "Bearbeitungszeit": processing_time[i],
            })

    schedule = pd.DataFrame(rows).sort_values(["Station", "Start"])
    display(schedule)
else:
    print("Keine optimale Lösung gefunden.")